# Mobile App for Lottery Addiction

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

What is the probability of winning the big prize with a single ticket?
What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?



In [1]:
# Create a function that computes the factorial of a number (n):
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

''' Create a function which takes in two inputs and outputs the 
number of combinations when we're taking only k objects from
a group of n objects.'''

def combinations(n, k):
    f_n = factorial(n)
    den = factorial(k) * factorial(n - k)
    return f_n / den


In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win. For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

Inside the app, the user inputs six different numbers from 1 to 49.
Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

We start by calculating the number of possible outcomes. There are 49 possible numbers, and six numbers are sampled without replacement. Since the order of the numbers doesn't matter, we are dealing with combinations.


In [2]:
''' Create a function which takes in a list of six unique numbers 
and prints the probability of winning'''
def one_ticket_probability(element):
    total_number_outcomes = combinations(49, 6)
    # Calculate the probability of winning for one-ticket
    p_win = 1 / total_number_outcomes
    perc = p_win * 100
    print("With a single ticket, you have {:.7f}% chance of winning".format(perc))
    print("In other words, you have 1 chance over {:,} to win the big prize.".format(total_number_outcomes))


We now test a bit the function on two different outputs.

In [3]:
test_input_1 = [2, 43, 22, 23, 11, 5]
one_ticket_probability(test_input_1)

With a single ticket, you have 0.0000072% chance of winning
In other words, you have 1 chance over 13,983,816.0 to win the big prize.


 For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.
Next, we'll focus on exploring the historical data coming from the Canada 6/49 lottery.
The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn in the following six columns:

- NUMBER DRAWN 1
- NUMBER DRAWN 2
- NUMBER DRAWN 3
- NUMBER DRAWN 4
- NUMBER DRAWN 5
- NUMBER DRAWN 6

Let's now write some code to open this data set and get familiar with its structure.

In [4]:
# Open the csv file to read the datas
import pandas as pd
loto_data = pd.read_csv('649.csv')
# Print the number of rowas and columns
loto_data.shape

(3665, 11)

In [5]:
# Print the first three rows
loto_data.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [6]:
# Print the last three rows
loto_data.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


In [5]:
'''Write a function that takes as input a row of the lottery 
dataframe and returns a set containing all the six winning 
numbers'''
def extract_numbers(row):
    data_row = loto_data[loto_data['DRAW NUMBER'] == row]
    numbers = set()
    columns = ['NUMBER DRAWN 1', 'NUMBER DRAWN 2', 'NUMBER DRAWN 3', 'NUMBER DRAWN 4', 'NUMBER DRAWN 5', 'NUMBER DRAWN 6']
    interest = data_row[columns]
    for i in range(6):
        number = interest.iloc[0, i]
        numbers.add(number)
    return numbers
extract_numbers(1)

{3, 11, 12, 14, 41, 43}

In [6]:
# Create a serie containing all the sets of combinations
SETS = loto_data['DRAW NUMBER'].apply(extract_numbers)
SETS = pd.Series(SETS)
SETS.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
Name: DRAW NUMBER, dtype: object

In [7]:
'''Write a function that compares the user's combination 
with the historical combinations.'''
def check_historical_occurence(user_list, historical_data):
    user_set = set()
    for num in user_list:
        user_set.add(num)
    results = (historical_data == user_set)
    counts = 0
    for x in results:
        if x is True:
            counts += 1
    print("Your combination occured {n} time(s) in the past".format(n=counts))
    print(one_ticket_probability(user_list))

We test the function.

In [10]:
test_input_3 = [33, 36, 37, 39, 8, 41]
check_historical_occurence(test_input_3, SETS)

Your combination occured 1 time(s) in the past
With a single ticket, you have 0.0000072% chance of winning
In other words, you have 1 chance over 13,983,816.0 to win the big prize.
None


So far, we wrote two functions:

- one_ticket_probability() — calculates the probability of winning the big prize with a single ticket

- check_historical_occurrence() — checks whether a certain combination has occurred in the Canada lottery data set
Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).

- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).

- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [21]:
''' Create a function that prints the probability of winning the big 
prize depending on the number of different tickets played'''
def multi_ticket_probability(number_of_tickets):
    total_number_outcomes = combinations(49, 6)
    if 1 <= number_of_tickets <= 13983816:
        # Calculate the probability of winning for n tickets
        number_of_tickets = int(number_of_tickets)
        p_win = number_of_tickets / total_number_outcomes
        perc = p_win * 100
        print("With {} ticket(s), you have {:.7f}% chance of winning".format(number_of_tickets, perc))
        print("In other words, you have {} chance(s) over {:,} to win the big prize.".format(number_of_tickets, total_number_outcomes))
    else:
        print('Unexpected number of tickets')


We test the function.

In [22]:
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for test_input in test_inputs:
    multi_ticket_probability(test_input)
    print('------------------------') # output delimiter

With 1 ticket(s), you have 0.0000072% chance of winning
In other words, you have 1 chance(s) over 13,983,816.0 to win the big prize.
------------------------
With 10 ticket(s), you have 0.0000715% chance of winning
In other words, you have 10 chance(s) over 13,983,816.0 to win the big prize.
------------------------
With 100 ticket(s), you have 0.0007151% chance of winning
In other words, you have 100 chance(s) over 13,983,816.0 to win the big prize.
------------------------
With 10000 ticket(s), you have 0.0715112% chance of winning
In other words, you have 10000 chance(s) over 13,983,816.0 to win the big prize.
------------------------
With 1000000 ticket(s), you have 7.1511238% chance of winning
In other words, you have 1000000 chance(s) over 13,983,816.0 to win the big prize.
------------------------
With 6991908 ticket(s), you have 50.0000000% chance of winning
In other words, you have 6991908 chance(s) over 13,983,816.0 to win the big prize.
------------------------
With 13983816

So far, we wrote three main functions:

- one_ticket_probability() — calculates the probability of winning the big prize with a single ticket

- check_historical_occurrence() — checks whether a certain combination has occurred in the Canada lottery data set

- multi_ticket_probability() — calculates the probability for any number of tickets between 1 and 13,983,816

Now, we're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

Inside the app, the user inputs:
- six different numbers from 1 to 49; and
- an integer between 2 and 5 that represents the number of winning numbers expected

Our function prints information about the probability of having the inputted number of winning numbers.

In [27]:
''' Create a function which takes in an integer between 2 and 5 
and prints information about the chances of winning depending on 
the value of that integer'''
def probability_less_6(n):
    n = int(n)
    # Calculate the number of n-number combinations
    num_combi = combinations(6, n)
    # Calculate the number of successful outcomes
    multiplier = combinations(49-n, 6-n)
    success_outcomes = num_combi * multiplier
    # Calculate the number of possible outcomes
    total_outcomes = combinations(49, 6)
    # Calculate the probability 
    p_win = success_outcomes / total_outcomes
    perc = p_win * 100
    simplified_combi = round(1/p_win)
    print("You have {p}% chance of having {n} winning numbers.".format(p=perc, n=n))
    print("In other words, you have 1 in {:,} chances to win.".format(simplified_combi))

We test the function.

In [28]:
for test_input in [2, 3, 4, 5]:
    probability_less_6(test_input)
    print('--------------------------') # output delimiter

You have 19.132653061224488% chance of having 2 winning numbers.
In other words, you have 1 in 5 chances to win.
--------------------------
You have 2.1710811984368217% chance of having 3 winning numbers.
In other words, you have 1 in 46 chances to win.
--------------------------
You have 0.10619418905397496% chance of having 4 winning numbers.
In other words, you have 1 in 942 chances to win.
--------------------------
You have 0.0018878966942928882% chance of having 5 winning numbers.
In other words, you have 1 in 52,969 chances to win.
--------------------------
